In [795]:
import pandas as pd
import random as rnd
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn import datasets
from sklearn.metrics import r2_score, f1_score, mean_absolute_error, mean_squared_error, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

# Стекинг

In [670]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict

class MyStacking(BaseEstimator, ClassifierMixin):  
    """Стэкинг моделей scikit-learn"""

    def __init__(self, models, ens_model):
        """
        Инициализация инстанса
        models - базовые модели для стекинга
        ens_model - мета-модель
        """
        self.models = models
        self.ens_model = ens_model
        self.n = len(models)
        self.X_valid = None
        
    def fit(self, X, y=None, p=0.25, cv=3, err=0.001, random_state=None):
        """
        Обучение стекинга
        p - в каком отношении делить на обучение / тест
            если p = 0 - используем всё обучение!
        cv  (при p=0) - сколько фолдов использовать
        err (при p=0) - величина случайной добавки к метапризнакам
        random_state - инициализация генератора
            
        """
        if p > 0: # делим на обучение и тест
            # разбиение на обучение моделей и метамодели
            X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=p, random_state=random_state)
            
            # заполнение матрицы для обучения метамодели
            self.X_valid = np.zeros((X_valid.shape[0], self.n))
            for i, mdl in enumerate(self.models):
                mdl.fit(X_train, y_train)
                self.X_valid[:, i] = mdl.predict(X_valid)
                
            # обучение метамодели
            self.ens_model.fit(self.X_valid, y_valid)
            
        else: # используем всё обучение
            
            # для регуляризации - берём случайные добавки
            self.X_valid = err*np.random.randn(X.shape[0], self.n)
            
            for i, mdl in enumerate(self.models):
                # это oob-ответы алгоритмов
                self.X_valid[:, i] += cross_val_predict(mdl, X, y, cv=cv, n_jobs=-1, method='predict')
                # но сам алгоритм надо настроить
                mdl.fit(X, y)
            
            # обучение метамодели
            self.ens_model.fit(self.X_valid, y)  
            
        return self


    def predict(self, X, y=None):
        """
        Предсказание стекингом
        """
        # заполение матрицы для мета-классификатора
        X_meta = np.zeros((X.shape[0], self.n))
        
        for i, mdl in enumerate(self.models):
            X_meta[:, i] = mdl.predict(X)
        
        return self.ens_model.predict(X_meta)


# Решение задачи стекингом

In [671]:
data = datasets.load_breast_cancer()
X = data["data"]
X = StandardScaler().fit_transform(X)
y = data["target"]


train_X, test_X, train_y, test_y = train_test_split(X, y, train_size=0.7)

In [672]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [673]:
mdls = [
    GaussianNB(var_smoothing=0.08),
    SVC(kernel="rbf", C=0.5),
    LogisticRegression(penalty="l2", C=0.1),
    KNeighborsClassifier(n_neighbors=9, metric="minkowski", algorithm="kd_tree")    
]

final_mdl = SVC(kernel="rbf", C=0.5)

In [674]:
stacking = MyStacking(mdls, final_mdl)
stacking.fit(train_X, train_y, p=0.25, cv=5, random_state=42)

MyStacking(ens_model=SVC(C=0.5),
           models=[GaussianNB(var_smoothing=0.08), SVC(C=0.5),
                   LogisticRegression(C=0.1),
                   KNeighborsClassifier(algorithm='kd_tree', n_neighbors=9)])

In [675]:
pred_y = stacking.predict(test_X)

In [676]:
f1_score(test_y, pred_y)

0.9753694581280787

In [677]:
for mdl in stacking.models:
    print(f1_score(test_y, mdl.predict(test_X)))

0.9468599033816424
0.9705882352941176
0.9705882352941176
0.9611650485436893


# Беггинг

In [794]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.base import clone

class MyBagging(BaseEstimator, ClassifierMixin):  
    """Беггинг моделей scikit-learn"""

    def __init__(self, 
                 model=LinearRegression(), 
                 n_estimators=100, 
                 type_of_task="regression", 
                 thershold=0.5, 
                 object_selection=1.0, 
                 feature_selection=1.0, 
                 random_state=None):
        """
        Инициализация инстанса
        model - базовая модель
        n_estimators - число моделей в беггинге
        type_of_task - тип решаемой задачи
            regression - регрессия; прогноз - среднее
            classification_binary_class - классификация; прогноз - класс {0, 1} среднее алгоритмов по порогу threshold
            classification_binary_proba - классификация; прогноз - вероятность 0...1 среднее алгоритмов
            
        """
        self.model = model
        self.models = []
        self.n_estimators = n_estimators
        if type_of_task not in ["regression", "classification_binary_class", "classification_binary_proba"]:
            raise ValueError(f"wrong type of task {type_of_task}\n expected: regression, " + 
                             "classification_binary_class, " +  
                             "classification_binary_proba")
        self.column_indexes_of_models = []
        self.type_of_task = type_of_task
        self.object_selection = object_selection
        self.feature_selection = feature_selection
        self.random_state = random_state
        
    def fit(self, X, y=None):
        """
        Обучение стекинга
        object_selection - доля объектов, используемая при обучении каждой модели
        feature_selection - доля признаков, используемая при обучении каждой модели
        random_state - инициализация генератора
        """
        row_index = np.array(range(X.shape[0]))
        col_index = np.array(range(X.shape[1]))
        
        for num_of_mdl in range(self.n_estimators):
            np.random.shuffle(row_index)
            np.random.shuffle(col_index)
            
            current_row_index = row_index[:int(len(row_index) * self.object_selection)]
            current_col_index = col_index[:int(len(col_index) * self.feature_selection)]
            self.column_indexes_of_models.append(current_col_index)
            
            mdl = clone(self.model)
            self.models.append(mdl.fit(X[current_row_index, :][:,current_col_index], y[current_row_index]))
            
        return self


    def predict(self, X, y=None):
        """
        Предсказание стекингом
        """
        res = []
        for i in range(len(self.models)):
            mdl = self.models[i]
            res.append(mdl.predict(X[:, self.column_indexes_of_models[i]]))
            
        if self.type_of_task == "regression":
            return np.mean(res, axis=0)
        elif self.type_of_task == "classification_binary_class":
            return np.mean(res, axis=0) > self.thershold
        elif self.type_of_task == "classification_binary_proba":
            return np.mean(res, axis=0)
        else:
            raise ValueError(f"wrong type of task {type_of_task}\n expected: regression, " +
                             "classification_binary_class, " +
                             "classification_binary_proba")

In [758]:
data = datasets.load_boston()
X = data["data"]
X = StandardScaler().fit_transform(X)
y = data["target"]
train_X, test_X, train_y, test_y = train_test_split(X, y, train_size=0.7)

In [760]:
bagging = MyBagging(DecisionTreeRegressor(max_depth=4), n_estimators=2)
bagging.fit(train_X, train_y)

mean_absolute_error(test_y, bagging.predict(test_X))

5.709765559681018

In [784]:
mdls = [DecisionTreeRegressor(max_depth=10).fit(train_X, train_y) for i in range(200)]
res = [mdl.predict(test_X) for mdl in mdls]
mean_absolute_error(test_y, np.mean(res, axis=0))

4.103374061392708

In [779]:
mdl = RandomForestRegressor(n_estimators=200, max_depth=10)
mdl.fit(train_X, train_y)
mean_absolute_error(test_y, mdl.predict(test_X))

3.0481806842186

In [793]:
from sklearn.ensemble import BaggingRegressor

regr = BaggingRegressor(base_estimator=DecisionTreeRegressor(max_depth=10),
                        n_estimators=1000, random_state=0).fit(train_X, train_y)
mean_absolute_error(test_y, regr.predict(test_X))

2.989022384650164